# standard imports

In [1]:
import dataiku
from dataiku import pandasutils as pdu
import pandas as pd
import os
import numpy as np

# langchain imports

In [2]:
from langchain import OpenAI

from langchain.agents import create_pandas_dataframe_agent
from langchain.chat_models import ChatOpenAI
from langchain.agents.agent_types import AgentType

# Open AI API Key

In [3]:
os.environ["OPENAI_API_KEY"] = '[YOUR API KEY HERE]'

# Loading DSS Dataset

In [4]:
jobs_dataset = dataiku.Dataset("dss_jobs")
df = jobs_dataset.get_dataframe()

# Creating LangChain Pandas Agent

In [5]:
agent = create_pandas_dataframe_agent(
    ChatOpenAI(temperature=0, model="gpt-3.5-turbo-0613"),
    df,
    verbose=True,
    agent_type=AgentType.OPENAI_FUNCTIONS
)

distutils Version classes are deprecated. Use packaging.version instead.
distutils Version classes are deprecated. Use packaging.version instead.


# Asking Questions

In [6]:
agent.run("what kinds of data does this dataset contain?")



> Entering new AgentExecutor chain...
Based on the provided dataframe, the dataset contains the following columns:

1. job_project_key: The project key of the job.
2. job_id: The ID of the job.
3. time_start: The start time of the job.
4. time_end: The end time of the job.
5. stable_state: Indicates whether the job is in a stable state.
6. state: The current state of the job.
7. warnings_count: The number of warnings associated with the job.
8. scenario_project_key: The project key of the scenario.
9. scenario_id: The ID of the scenario.
10. scenario_run_id: The run ID of the scenario.
11. step_id: The ID of the step.
12. step_run_id: The run ID of the step.
13. job_def_type: The type of job definition.
14. job_def_trigger_type: The trigger type of the job definition.
15. job_def_initiator: The initiator of the job definition.
16. job_def_initiated: The time when the job definition was initiated.
17. job_def_refresh_metastore: Indicates whether the job definition refreshes the metast

'Based on the provided dataframe, the dataset contains the following columns:\n\n1. job_project_key: The project key of the job.\n2. job_id: The ID of the job.\n3. time_start: The start time of the job.\n4. time_end: The end time of the job.\n5. stable_state: Indicates whether the job is in a stable state.\n6. state: The current state of the job.\n7. warnings_count: The number of warnings associated with the job.\n8. scenario_project_key: The project key of the scenario.\n9. scenario_id: The ID of the scenario.\n10. scenario_run_id: The run ID of the scenario.\n11. step_id: The ID of the step.\n12. step_run_id: The run ID of the step.\n13. job_def_type: The type of job definition.\n14. job_def_trigger_type: The trigger type of the job definition.\n15. job_def_initiator: The initiator of the job definition.\n16. job_def_initiated: The time when the job definition was initiated.\n17. job_def_refresh_metastore: Indicates whether the job definition refreshes the metastore.\n18. job_def_out

In [7]:
agent.run("when was the most recent FAILED job? what was the project key of that job?")



> Entering new AgentExecutor chain...

Invoking: `python_repl_ast` with `{'query': "df[df['state'] == 'FAILED']['time_end'].max()"}`


2023-09-17 12:00:30.748000+00:00
Invoking: `python_repl_ast` with `{'query': "df[(df['state'] == 'FAILED') & (df['time_end'] == '2023-09-17 12:00:30.748000+00:00')]['job_project_key'].values[0]"}`


DSS_LOGGINGThe most recent FAILED job occurred on September 17, 2023, at 12:00:30.748 UTC. The project key of that job was "DSS_LOGGING".

> Finished chain.


'The most recent FAILED job occurred on September 17, 2023, at 12:00:30.748 UTC. The project key of that job was "DSS_LOGGING".'

# PDF Loader

In [8]:
from langchain.document_loaders import PyPDFLoader

# pdfs from https://www.loc.gov/item/2020445568/
handle = dataiku.Folder("pdfs")
paths = handle.list_paths_in_partition()

pages = []
loaders = []

# indexing all the PDFs in our folder
for path in paths:
    loader = PyPDFLoader(handle.get_path() + path)
    pgs = loader.load_and_split()
    
    pages = np.append(pages, pgs)
    loaders.append(loader)
    print(f'indexed {len(pages)} pages in {path}')

indexed 4 pages in /2F73J4NP2YHKVISKHDIDJ7RGPDKTQZ7D.pdf
indexed 7 pages in /2A2C2V4WI5YRDJHR26XUD4IAULIYGTMA.pdf
indexed 11 pages in /2ED27NR7CISW7J4PHXXBZ6OFPVDFHMFB.pdf
indexed 12 pages in /2EDEPZ4VHTLPTWSZR6FAVUJ3B2ZVSIPS.pdf


In [9]:
from langchain.indexes import VectorstoreIndexCreator

index = VectorstoreIndexCreator().from_loaders(loaders)

The ``declarative_base()`` function is now available as sqlalchemy.orm.declarative_base(). (deprecated since: 2.0) (Background on SQLAlchemy 2.0 at: https://sqlalche.me/e/b8d9)


In [26]:
query = "What date did the USCIRF plan to leave for India?"
index.query(query)

' June 12, 2009'

In [19]:
query = "Who does the OIG advise?"
index.query(query)

' The OIG advises DOJ-OPR of the results of their investigations.'

In [8]:
query = "What level of education is required for Broadcast Technicians?"
index.query(query)

" Most occupations in this zone require training in vocational schools, related on-the-job experience, or an associate's degree. Some may require a bachelor's degree."